In [1]:
from hstar import process
from hstar import trilinear
from hstar import datacuts

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras import Model, backend

2024-10-30 14:08:45.549937: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
sample = process.Sample(weight='wt', 
    amplitude = process.Basis.SBI, components = {
    process.Basis.SBI: 'msq_sbi_sm',
    process.Basis.SIG: 'msq_sig_sm',
    process.Basis.BKG: 'msq_bkg_sm',
    process.Basis.INT: 'msq_int_sm'
  })

sample.open(csv = [
  '../ggZZ4e_all.csv',
  '../ggZZ4m_all.csv',
  '../ggZZ2e2m_all.csv'
  ], xs=[1.4783394, 0.47412769, 0.47412769], lumi=3000., k=1.83, nrows=100000
)

0        6.917263e-12
1        4.352699e-10
2        1.308438e-09
3        1.289731e-09
4        1.347206e-13
             ...     
99995    1.513117e-10
99996    4.439589e-09
99997    1.220276e-09
99998    1.993417e-09
99999    9.613625e-10
Name: msq_sbi_sm, Length: 300000, dtype: float64
0        7.188679e-14
1        3.480814e-11
2        1.946946e-11
3        1.735510e-11
4        6.937647e-15
             ...     
99995    3.494904e-11
99996    2.558532e-11
99997    3.352285e-11
99998    1.282424e-10
99999    7.091425e-11
Name: msq_sig_sm, Length: 300000, dtype: float64
0        7.326895e-12
1        4.924589e-10
2        1.370247e-09
3        1.360140e-09
4        1.353117e-13
             ...     
99995    1.554967e-10
99996    4.562270e-09
99997    1.308377e-09
99998    2.165222e-09
99999    1.049910e-09
Name: msq_bkg_sm, Length: 300000, dtype: float64
0       -4.813897e-13
1       -9.161463e-11
2       -8.107657e-11
3       -8.776563e-11
4       -1.676082e-14
             ... 

In [3]:
evt = datacuts.Events4l(sample)

kin_variables = evt.get_kinematics()

where_nan = np.where(np.isnan(kin_variables))


print(kin_variables, kin_variables.shape)

/home/max/Uni/WS24/BachelorArbeit/higgs-offshell-interpretation/hstar/datacuts.py:242: RuntimeWarning: invalid value encountered in divide
  phi = z1.dot(n12.cross(n34))/np.abs(z1.dot(n12.cross(n34)))*np.arccos(-n12.dot(n34))
/home/max/Uni/WS24/BachelorArbeit/higgs-offshell-interpretation/hstar/datacuts.py:242: RuntimeWarning: invalid value encountered in arccos
  phi = z1.dot(n12.cross(n34))/np.abs(z1.dot(n12.cross(n34)))*np.arccos(-n12.dot(n34))
/home/max/Uni/WS24/BachelorArbeit/higgs-offshell-interpretation/hstar/datacuts.py:243: RuntimeWarning: invalid value encountered in arccos
  illegal_inds = np.where(np.isnan(np.arccos(-n12.dot(n34))))[0]


[(-0.74512123, -0.66691146,  0.00484323)
 ( 0.90303732,  0.10368025,  0.41686209)
 ( 0.74217209, -0.66865606,  0.04560333) ...
 (-0.45940078, -0.68314908, -0.56767796)
 (-0.50510544, -0.80608794, -0.30836785)
 ( 0.77281874, -0.32284716,  0.54637066)]
[(-0.74512123, -0.66691146,  0.00484323)
 ( 0.90303732,  0.10368025,  0.41686209)
 ( 0.74217209, -0.66865606,  0.04560333) ...
 (-0.45940078, -0.68314908, -0.56767796)
 (-0.50510544, -0.80608794, -0.30836785)
 ( 0.77281874, -0.32284716,  0.54637066)]
[[ 0.96633291  0.9907067   0.58420921 ... -1.22078896 90.72999361
  94.15907615]
 [-0.99871125  0.99720293  0.99234008 ...  0.28684574 90.57756562
  93.74116556]
 [ 0.97583909  0.9823587   0.9677601  ... -2.73044289 93.07948836
  90.44019454]
 ...
 [-0.14328388 -0.27324394  0.63449089 ... -0.94574492 90.30534808
  88.61756136]
 [-0.9940422   0.98857818  0.99953491 ... -2.01282493 92.56397478
  91.86507616]
 [ 0.83711788  0.83430643 -0.504289   ... -0.10824956 92.27354723
  93.33937647]] (29225

In [4]:
c6_values = np.linspace(-10,10,21)

In [5]:
c6_mod = trilinear.Modifier(c6_values = [-5,-1,0,1,5], c6_amplitudes = ['msq_sbi_c6_6', 'msq_sbi_c6_10', 'msq_sbi_c6_11', 'msq_sbi_c6_12', 'msq_sbi_c6_16'])
(coeffs, c6_weights) = c6_mod.modify(sample=evt.sample, c6=c6_values, tensorize=False)

In [6]:
data = []

for i in range(len(c6_values)):
    c6 = c6_values[i]

    # {kin}, c6, label, weight
    data_part_sig = np.append(np.append(kin_variables, np.ones((kin_variables.shape[0],1))*c6, axis=1), np.ones((kin_variables.shape[0],1)), axis=1)
    sig_weights = c6_weights.T[i][:,np.newaxis]
    sig_weights *= 1/np.sum(sig_weights)

    data_part_sig = np.append(data_part_sig, sig_weights, axis=1)

    data_part_bkg = np.append(np.append(kin_variables, np.ones((kin_variables.shape[0],1))*c6, axis=1), np.zeros((kin_variables.shape[0],1)), axis=1)
    bkg_weights = np.array(evt.df['wt'])[:,np.newaxis]
    bkg_weights *= 1/np.sum(bkg_weights)

    data_part_bkg = np.append(data_part_bkg, bkg_weights, axis=1)

    data_part = np.append(data_part_sig, data_part_bkg, axis=0)

    data.append(data_part)


data = np.reshape(np.array(data), (np.array(data).shape[0]*np.array(data).shape[1], np.array(data).shape[2]))
print(data, data.shape)


[[ 9.66332908e-01  9.90706697e-01  5.84209211e-01 ... -1.00000000e+01
   1.00000000e+00  4.04819288e-06]
 [-9.98711253e-01  9.97202928e-01  9.92340082e-01 ... -1.00000000e+01
   1.00000000e+00  7.53960947e-06]
 [ 9.75839094e-01  9.82358700e-01  9.67760103e-01 ... -1.00000000e+01
   1.00000000e+00  1.77111083e-05]
 ...
 [-1.43283881e-01 -2.73243938e-01  6.34490892e-01 ...  1.00000000e+01
   0.00000000e+00  2.65801633e-07]
 [-9.94042201e-01  9.88578178e-01  9.99534906e-01 ...  1.00000000e+01
   0.00000000e+00  8.06941886e-06]
 [ 8.37117879e-01  8.34306435e-01 -5.04288996e-01 ...  1.00000000e+01
   0.00000000e+00  1.63848877e-07]] (12274668, 10)


In [7]:

def swish_activation(x, b=1):
    return x*backend.sigmoid(b*x)

get_custom_objects().update({'swish_activation': Activation(swish_activation)})


class C6_4l_clf(Model):
    def __init__(self):
        super().__init__()

        swish = Activation(swish_activation, name='Swish')

        self.dense1 = Dense(1000, activation=swish, input_dim=8, kernel_initializer='he_normal')
        self.dense2 = Dense(1000, activation=swish, kernel_initializer='he_normal')
        self.dense3 = Dense(1000, activation=swish, kernel_initializer='he_normal')
        self.dense4 = Dense(1000, activation=swish, kernel_initializer='he_normal')
        self.dense5 = Dense(1000, activation=swish, kernel_initializer='he_normal')
        self.out = Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        return self.out(x)

In [8]:
model = C6_4l_clf()

optimizer = Nadam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07
)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'], weighted_metrics=['binary_accuracy'])

2024-10-30 14:09:01.443612: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
